# 고려 사항
##### 모델 사이즈 : 원본 사이즈는 112 X 112
##### 표본의 크기 : Max size ( 타켓별 랜덤샘플링 할때의 최대 표본 크기)
##### 주성분 크기 : pca
##### svc 파라미터 : gamma, c 값 (그리드서치이용)

In [1]:
model_size = (56,56)
sample_size = 10
pca_size = 100
index_size = 125000

라이브러리 불러오기

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
import joblib
from numpy.linalg import norm
from numpy import dot

데이터 불러오기

In [4]:
def sampling_func(data, max_size):
    N = len(data)
    sample_n = len(data) 
    if sample_n > max_size:
        sample = data.take(np.random.permutation(N)[:max_size])
    else:
        sample = data.take(np.random.permutation(N)[:len(data)])
    return sample

ID_List = pd.read_table('/Users/heogeonhyeock/Downloads/inha_data/inha_data/ID_List.txt',
                       sep = " ",
                       names = ["name", "path"])

ID_List_sample = ID_List.groupby('name', group_keys = False
                                ).apply(sampling_func, max_size = sample_size).reset_index()


In [5]:
train_img = []
train_name = []

for index in range(index_size):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/" + ID_List_sample['path'][index]
    file = cv2.resize(cv2.imread(A,0), model_size).reshape(1,-1)[0]
    train_img.append(file) # 리스트에 이미지 추가
    train_name.append(ID_List_sample['name'][index])

train_img = np.array(train_img)
train_name = np.array(train_name)

train = dict(data = train_img,
            target = train_name)

모델

In [6]:
# 모델 생성
pca = RandomizedPCA(n_components = pca_size, whiten=True)
svc = SVC(kernel = "rbf", C= 5, gamma=0.01,
        probability=True ,cache_size=10000)
model = make_pipeline(pca,svc)

In [7]:
"""
# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

param_grid = {'svc__C' : [1,5,10,15],
              'svc__gamma' : [0.001,0.005,0.01,0.05]}

grid = GridSearchCV(model, param_grid, cv=3 ,n_jobs=6)
grid.fit(train['data'], train['target'])

model = grid.best_estimator_
"""

# 모델 훈련
model.fit(train['data'], train['target'])

Pipeline(steps=[('pca', PCA(n_components=100, whiten=True)),
                ('svc',
                 SVC(C=5, cache_size=10000, gamma=0.01, probability=True))])

In [8]:
# 모델 저장
joblib.dump(model, './Inha_svc_8.14.pkl')

['./Inha_svc_8.14.pkl']

모델 적용

In [9]:
# 테스트 데이터
left_test_img = []
right_test_img = []

for index in range(6000):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/left_face_" + str(index) + ".jpg"
    B = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/right_face_" + str(index) + ".jpg" 
    
    file_left = cv2.resize(cv2.imread(A,0), model_size).reshape(1,-1)[0]
    
    file_right = cv2.resize(cv2.imread(B,0), model_size).reshape(1,-1)[0]

    left_test_img.append(file_left)
    right_test_img.append(file_right)


In [ ]:
# 정답 저장 코사인 유사도

predict = []

predict_left_prob = model.predict_proba(left_test_img)
predict_right_prob = model.predict_proba(right_test_img)

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
for i in range(len(predict_left_prob)):
    predict_cos = cos_sim(predict_left_prob[i],predict_right_prob[i])
    predict.append(predict_cos)

submission = pd.read_csv("/Users/heogeonhyeock/Downloads/inha_data/inha_data/sample_submission.csv")
submission["answer"] = predict[:]

path = '/Users/heogeonhyeock/Downloads/inha_data/inha_data'
os.chdir(path)
submission.to_csv("submission.csv", index = False)

submission